In [1]:
from dataset import MyDataset
from torch.utils.data import DataLoader
from model import NetForYolo
import torch 
import torch.nn as nn
from tqdm import tqdm
categories = ["bus", "cat", "pizza"]

ynet = NetForYolo(depth=2)
y_params = torch.load('/Users/akshita/Documents/Acads/models/final_train.pt', map_location=torch.device('cpu'))
ynet.load_state_dict(y_params)

<All keys matched successfully>

In [107]:
len(list(ynet.parameters()))

56

In [2]:
batch_size = 4
# train_data = MyDataset(categories=categories, split='train', manifest_path='./manifests', mac=True)
val_data = MyDataset(categories=categories, split='val', manifest_path='./manifests', mac=True)
# val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

# next(iter(val_loader))

def validation(val_loader, ynet, yolo_interval=32):
    device = "cuda" if torch.cuda.is_available() == True else "cpu"
    print(device)
    ynet = ynet.to(device)
    ynet.eval()
    predicted_bboxes = []
    total_true_bboxes = []
    total_val_classes = []
    true_val_classes = []
    with torch.no_grad():
        # for data in val_loader:
        for d in tqdm(range(len(val_loader))):
        # for d in tqdm(range(10)):
            inp, t_l, true_bboxes = val_data.__getitem__(d)
            # data
        #     inp, t_l, true_bboxes = data
            inp = inp.to(device)

            out = ynet(inp.unsqueeze(0))
            out = out.view(-1, 64, 5, 9)

        #     # get cells with top 5 highest values in the first element of the predicted yolo_vectors
        #     # to achieve this, first we get the max value of anchor boxes for each cell
        #     # then we sort the values in descending order and get the top 5 cells
            pred_vals = out[:, :, :, 0]
            pred_vals, _ = torch.max(pred_vals, dim=-1)
            sorted_cells = torch.argsort(pred_vals, descending=True, dim=-1)

            top5preds = torch.zeros((out.shape[0], 64, 9))

            for i in range(out.shape[0]):
                each_batch = out[i, sorted_cells[i]]
                for j in range(each_batch.shape[0]):
                    temp = each_batch[j, :, 0]
                    args = torch.argmax(temp, dim=-1)
                    top5preds[i, j] = each_batch[j, args]

            pred_classes = top5preds[:, :, 5:-1]
            pred_classes = nn.Softmax(dim=1)(pred_classes)
            pred_classes = torch.argmax(pred_classes, dim=-1)

            pred_regression_vec = top5preds[:, :, 1:5]
            del_x, del_y = pred_regression_vec[:, :, 0], pred_regression_vec[:, :, 1]
            h, w = pred_regression_vec[:, :, 2], pred_regression_vec[:, :, 3]

            h *= yolo_interval
            w *= yolo_interval
            cell_row_index = torch.div(sorted_cells, 8, rounding_mode="floor")
            cell_col_index = sorted_cells % 8

            yolo_offset = torch.ones_like(cell_row_index) * (yolo_interval / 2)
            bb_center_x = (
                cell_col_index * yolo_interval + yolo_offset + del_x * yolo_interval
            )
            bb_center_y = (
                cell_row_index * yolo_interval + yolo_offset + del_y * yolo_interval
            )

            bb_top_left_x = bb_center_x - torch.div(w, 2, rounding_mode="floor")
            bb_top_left_y = bb_center_y - torch.div(h, 2, rounding_mode="floor")

            valid_preds = []
            val_classes = []
            for i in range(bb_top_left_x.shape[0]):
                for j in range(bb_top_left_x.shape[1]):
                    if bb_top_left_x[i, j] < 0:
                        bb_top_left_x[i, j] = 0
                    if bb_top_left_y[i, j] < 0:
                        bb_top_left_y[i, j] = 0
                    if (
                        (h[i, j] > 256).any()
                        or (w[i, j] > 256).any()
                        # or ((w < 64).any() and (h < 64).any())
                    ):
                        # print(d)
                        continue
                    valid_preds.append(
                        torch.tensor(
                            [bb_top_left_x[i, j], bb_top_left_y[i, j], w[i, j], h[i, j]]
                        )
                    )
                    val_classes.append(pred_classes[i, j].item())

            if len(valid_preds):
                valid_preds = torch.stack(valid_preds, dim=-1)
                predicted_bboxes.append(valid_preds)
                # print(true_bboxes)
                total_true_bboxes.append(true_bboxes)
                total_val_classes.append(val_classes)
                # print(val_classes)
                true_val_classes.append(t_l)


    return predicted_bboxes, total_true_bboxes, total_val_classes, true_val_classes
    
predicted_bboxes, total_true_bboxes, total_val_classes, true_val_classes = validation(val_data, ynet)
# predicted_bboxes.shape

cpu


100%|██████████| 3494/3494 [11:12<00:00,  5.20it/s]   


In [3]:
import pickle

predicted_bboxes_np = []
for p in predicted_bboxes:
    predicted_bboxes_np.append(p.numpy())


with open('predicted_bboxes.pkl', 'wb') as f:
    pickle.dump(predicted_bboxes_np, f)
with open('total_true_bboxes.pkl', 'wb') as f:
    pickle.dump(total_true_bboxes, f)
with open('total_val_classes.pkl', 'wb') as f:
    pickle.dump(total_val_classes, f)
with open('true_val_classes.pkl', 'wb') as f:
    pickle.dump(true_val_classes, f)

In [50]:
import numpy as np
total_true_bboxes_np = []
for i in range(len(total_true_bboxes)):
    # print(np.array(total_true_bboxes[i]).shape)
    total_true_bboxes_np.append(np.array(total_true_bboxes[i]))

In [82]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

def validate_and_conf_matrix(t, p, categories, name="Net") -> None:
    cm = confusion_matrix(t, p)
    plt.figure()
    sns.heatmap(cm, annot=cm, xticklabels=categories, yticklabels=categories, fmt="g")
    plt.title(f"Confusion matrix for {name}, accuracy={accuracy_score(t, p)}")
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.savefig(f"./solutions/cm_{name}.png")

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

t, p = [], []
iou_tot = {}
bbx_tot = {}
labels = {}
# print(len(predicted_bboxes_np))
for i in range(len(predicted_bboxes_np)):
    iou = []
    bbx = []
    for j in range(total_true_bboxes_np[i].shape[0]):
        # print(predicted_bboxes_np[i][:, j])
        t_x, t_y, t_w, t_h = total_true_bboxes[i][j]
        x, y, w, h = predicted_bboxes_np[i][:, j]
        x = int(x)
        y = int(y)
        w = int(w)
        h = int(h)
        if ((w < 0 or h < 0) or (x+w > 256 or y+h > 256)
            or ((w < 64) and (h < 64))
            ):
            continue

        iou.append(bb_intersection_over_union([x, y, x+w, y+h], 
                                                 [t_x, t_y, t_x+t_w, t_y+t_h]))
        bbx.append(([x, y, x+w, y+h], [t_x, t_y, t_x+t_w, t_y+t_h]))
        # print(total_val_classes[i][j], true_val_classes[i])
        t.append(total_val_classes[i][j])
        p.append(true_val_classes[i])
    if len(iou):
        iou_tot[i] = iou
        bbx_tot[i] = bbx
        labels[i] = true_val_classes[i]
    
validate_and_conf_matrix(t, p, categories, name="yolo_net")


In [115]:
iou_tot

{16: [0.0],
 25: [0.2699979850896635],
 41: [0.3374531536560083],
 42: [0.0],
 47: [0.041848299912816043],
 48: [0.4473684210526316],
 49: [0.0],
 51: [0.3941227312013829],
 54: [0.5339805825242718],
 59: [0.4506405425772419],
 68: [0.29558541266794625],
 74: [0.11456825856246985],
 75: [0.1234512946064248],
 81: [0.0],
 84: [0.01597812005182093],
 86: [0.5168158156401708],
 94: [0.3538171349151196],
 99: [0.24540441176470587],
 110: [0.0],
 118: [0.0],
 130: [0.0],
 131: [0.19430950728660654],
 132: [0.10419161676646707],
 135: [0.10198340132226755],
 137: [0.21280276816608998],
 142: [0.0],
 147: [0.41464096116233584],
 155: [0.6007227948065854],
 158: [0.19020108043217288],
 162: [0.0],
 195: [0.0],
 221: [0.0],
 223: [0.6178462580426685],
 240: [0.0],
 265: [0.0],
 289: [0.418848167539267],
 291: [0.4],
 294: [0.0],
 306: [0.24330900243309003],
 318: [0.0],
 324: [0.0, 0.0],
 325: [0.3446327683615819],
 327: [0.22695035460992907],
 328: [0.5280496538553354],
 336: [0.57901085645355

In [119]:
import cv2
from PIL import Image

flag = 0
for i in list(iou_tot.keys()):
    # print(i, bb_tot[i])
    cnt = 0
    img = cv2.imread(val_data.clip_file_names(val_data.file_list[i]))
    # img = np.array(img).astype(np.uint8)
    # img = img.numpy()
    for j in range(len(iou_tot[i])):
        # if iou_tot[i][j] == 0:
        #     cnt += 1
        # print(i, cnt)
            # print("True")
            # print(i, j)
            img = cv2.rectangle(img, (bbx_tot[i][j][0][0], bbx_tot[i][j][0][1]),
                                (bbx_tot[i][j][0][2], bbx_tot[i][j][0][3]),
                                color=(0, 0, 255), thickness=2)
            
            img = cv2.rectangle(img, (bbx_tot[i][j][1][0], bbx_tot[i][j][1][1]),
                                (bbx_tot[i][j][1][2], bbx_tot[i][j][1][3]),
                                color=(0, 255, 0), thickness=2)
            
            img = cv2.putText(img, f"{categories[labels[i]]}", (bbx_tot[i][j][0][0], bbx_tot[i][j][0][1]-10),
                        color=(255, 0, 0), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, thickness=2)
            
    # if cnt > 1:
        # print(i, cnt, labels[i])
            cv2.imwrite(f"./pizza/{i}_true.png", img)
            # flag+=1
    # if flag == 2:
    #     break
        
            # break